In [ ]:
def algo_process(frame,range_zero_pad,peak_thresh,peak_slice,peak_width,peak_relevance,search_zone,bin2range):
    if not isinstance(frame,fr.Frame):
        print("Argument of algo_process() can only be a Frame instance")
        return
    #Fast time Windowing
    tmp_frame = frame.chirp
    window = 0*2*np.hamming(frame.sam_x_chir)+1
    tmp_frame = apply_window(tmp_frame, window,'fast')
    #Zero-pad & Fast FFT
    fast_fft_data_out = fftn(np.pad(tmp_frame,((0,range_zero_pad-frame.sam_x_chir),(0,0),(0,0))), axes=0)
    selected_chirp = np.abs(fast_fft_data_out[:,0,0])
    #Peak search
    search_zone = round(search_zone/bin2range)
    target_position,target_strength = peak_search(selected_chirp,'fast',peak_thresh,peak_width,peak_relevance,search_zone)
    target_info = alg_res.algo_result(target_position, target_strength,bin2range, peak_slice)
    #Target info collection
    for target in range(target_info.num_targets):
        spectrum_slice = peak_extraction(selected_chirp,target_position[target],target_info.radius,target_info.bin2range)
        target_info.collect_target_info(spectrum_slice)
    return fast_fft_data_out,target_info
    

In [ ]:
def apply_window(data_in, window, direction):
    if direction == 'fast':
        return (data_in.T*window).T
    if direction == 'slow':
        return (data_in*window)

In [ ]:
def peak_search(chirp,direction,thresh,peak_width,peak_relevance,search_zone):
    slices = round(len(chirp)/search_zone)
    peak_frequency = []
    peak_str = []
    for spectrum_slice in range(slices):
        fbc = np.argmax(chirp[spectrum_slice*search_zone:(spectrum_slice+1)*search_zone]) + spectrum_slice*search_zone
        fbl2 = fbc - 2
        fbl1 = fbc - 1
        fbr1 = fbc + 1
        fbr2 = fbc + 2
        if fbr2 >= len(chirp):
            continue
        pvl2 = chirp[fbl2]
        pvl1 = chirp[fbl1]
        pvc = chirp[fbc]
        pvr1 = chirp[fbr1]
        pvr2 = chirp[fbr2]
        if pvl2+peak_relevance < pvl1 and pvr1 > pvr2+peak_relevance and (pvc > thresh) and pvc >= pvl1 and pvc >= pvr1:
                peak_frequency.append((fbl2*pvl2 + fbl1*pvl1+ fbc*pvc + fbr1*pvr1 + fbr2*pvr2)/(pvl2 + pvl1 + pvc +pvr1 + pvr2))
                peak_str.append(pvc)
    return peak_frequency,peak_str

In [ ]:
def peak_extraction(chirp,idx,radius,b2r):
    idx_slice = np.arange(round(idx) - round(radius/b2r),round(idx) + round(radius/b2r))
    return idx_slice,chirp[idx_slice]